In [2]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from catboost import CatBoostClassifier
# import training_preprocess



In [3]:
# using the preprocessed data
from data_reprocessing import train_to_df
train_file = 'new_train.csv'  
dataset = train_to_df(train_file)
# Display the first few rows of the dataset to verify the preprocessing
dataset.head()

,ip,app,device,os,channel,is_attributed,day,hour,minute,second
0,122044,20,1,13,478,0,7,13,51,27
1,191847,26,1,11,477,0,9,1,10,13
2,29785,3,1,19,466,0,7,7,10,3
3,194772,18,1,19,107,0,6,16,28,59
4,119369,2,1,19,469,0,9,10,12,23


In [4]:
# Split the data into features and target
X = dataset.drop('is_attributed', axis=1)
y = dataset['is_attributed']

# Split the dataset into 80% training and 20% validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Data split into training and validation sets.")
print(f"Training set shape: {X_train.shape}")
print(f"Validation set shape: {X_val.shape}")


Data split into training and validation sets.
Training set shape: (118338489, 9)
Validation set shape: (29584623, 9)


In [5]:
# Initialize CatBoost Classifier
catboost_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    eval_metric='AUC',
    random_seed=42,
    verbose=100
)

print("Starting model training...")

# Train the CatBoost model
catboost_model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50)

print("Model training completed.")

# Save the trained CatBoost model to a JSON file
catboost_model.save_model('catboost_model.json', format='json')

print("Model saved as 'catboost_model.json'.")


Starting model training...
0:	test: 0.5552535	best: 0.5552535 (0)	total: 6.22s	remaining: 1h 43m 36s
100:	test: 0.9422230	best: 0.9422230 (100)	total: 6m 32s	remaining: 58m 16s
200:	test: 0.9447417	best: 0.9487388 (190)	total: 13m 3s	remaining: 51m 56s
300:	test: 0.9545953	best: 0.9545953 (300)	total: 19m 34s	remaining: 45m 28s
400:	test: 0.9578994	best: 0.9578994 (400)	total: 26m 9s	remaining: 39m 4s
500:	test: 0.9600230	best: 0.9600230 (500)	total: 32m 40s	remaining: 32m 32s
600:	test: 0.9612219	best: 0.9612219 (600)	total: 39m 11s	remaining: 26m 1s
700:	test: 0.9618104	best: 0.9618104 (700)	total: 45m 47s	remaining: 19m 31s
800:	test: 0.9626756	best: 0.9626756 (800)	total: 52m 27s	remaining: 13m 1s
900:	test: 0.9633924	best: 0.9633928 (899)	total: 59m 2s	remaining: 6m 29s
999:	test: 0.9637623	best: 0.9637623 (999)	total: 1h 5m 29s	remaining: 0us

bestTest = 0.9637623442
bestIteration = 999

Model training completed.
Model saved as 'catboost_model.json'.
